# M1M3 cell learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')
FATABLE = M1M3FATable.FATABLE

In [ ]:
type(client)

In [ ]:
# Times of bump test
start = Time("2023-04-17T10:00:00", scale='utc')
end = Time("2023-04-17T11:15:00", scale='utc')

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

## Given an actuator ID, this plots the bump test result

In [ ]:
async def plotBumpTestResults(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    # The pass/fail results are actually in the next test.
    lastThisBumpIndex = bumps[bumps['actuatorId']==id].last_valid_index()
    passFail = bumps.iloc[bumps.index.get_loc(lastThisBumpIndex)+1]
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    if passFail[primaryBump] == 6:
        plt.text(2.0, 350.0, "PASSED", color='g')
    elif passFail[primaryBump] == 7:
        plt.text(2.0, 350.0, "FAILED", color='r')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 14.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        if passFail[secondaryBump] == 6:
            plt.text(2.0, 350.0, "PASSED", color='g')
        elif passFail[secondaryBump] == 7:
            plt.text(2.0, 350.0, "FAILED", color='r')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
fig = plt.figure(figsize=(10,5))
await plotBumpTestResults(fig, bumps, 227)

## Now let's look at more of them

In [ ]:
# Times of bump test
start = Time("2020-06-01T00:00:00", scale='utc')
#end = Time("2022-06-01T00:00:00", scale='utc')
end = Time("2023-04-20T00:00:00", scale='utc')

In [ ]:
manyBumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
len(manyBumps)

In [ ]:
async def plotMultipleBumpTestResults(fig, manyBumps, id):
    theseBumps = manyBumps[manyBumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None
    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Multiple bump tests Actuator ID {id}", fontsize=18)

    # Now find the separate tests
    times = theseBumps['timestamp'].values
    startTimes = []
    endTimes = []
    for i, time in enumerate(times):
        if i == 0:
            startTimes.append(time)
            continue
        if (time - times[i-1]) > 60.0:
            startTimes.append(time)
            endTimes.append(times[i-1])
    endTimes.append(times[-1])
    numPlots = 0
    passCount = 0
    failCount = 0
    for i in range(len(startTimes)):
        startTime = startTimes[i]
        endTime = endTimes[i]
        thisBump = theseBumps[(theseBumps['timestamp'] >= startTime) & (theseBumps['timestamp'] <= endTime)]
        try:
            numPlots += 1
            plt.subplot(1,2,1)
            plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
            plotEnd = plotStart + 14.0
            start = Time(plotStart, format='unix_tai', scale='tai')
            end = Time(plotEnd, format='unix_tai', scale='tai')
            forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], \
                                                     start.utc, end.utc)
            times = forces['timestamp'].values
            t0 = times[0]
            times -= t0
            plotStart -= t0
            plotEnd -= t0
            lastIndex = bumps[bumps['actuatorId']==id].last_valid_index()
            passFail = bumps.iloc[bumps.index.get_loc(lastIndex)+1][primaryBump]
            #print(i, passFail)
            if passFail == 6:
                passCount += 1
                #print("PASSED")
            elif passFail == 7:
                failCount += 1
                #print("FAILED")

            plt.title("Primary   Z")
            plt.plot(times, forces[primaryForce].values)
            plt.xlim(plotStart, plotEnd)
            plt.ylim(-400,400)
            plt.xlabel("Time (seconds)")
            plt.ylabel("Force (nt)")
            plt.subplot(1,2,2)
            if secondaryName is not None:
                plt.title(f"Secondary   {secondaryName}")
                plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
                plotEnd = plotStart + 14.0
                start = Time(plotStart, format='unix_tai', scale='tai')
                end = Time(plotEnd, format='unix_tai', scale='tai')
                forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], \
                                                         start.utc, end.utc)
                times = forces['timestamp'].values
                t0 = times[0]
                times -= t0
                plotStart -= t0
                plotEnd -= t0
                plt.plot(times, forces[secondaryForce].values)
                plt.xlim(plotStart, plotEnd)
                plt.ylim(-400,400)
                plt.xlabel("Time (seconds)")
                plt.ylabel("Force (nt)")
            else:
                plt.title("No Secondary")
                plt.xticks([])
                plt.yticks([])
        except:
            continue
            
    plt.subplot(1,2,1)
    plt.text(2.0, 350, f"{numPlots} tests, {passCount} passed, {failCount} failed")
    plt.subplot(1,2,2)
    plt.text(2.0, 350, f"{numPlots} tests, {passCount} passed, {failCount} failed")
    return

In [ ]:
id = 227
fig = plt.figure(figsize=(10,5))
await plotMultipleBumpTestResults(fig, manyBumps, id)
#plt.savefig(f"/scratch/cslage/m1m3_data/Bump_Test_{id}_17Apr23.png")

## Now generate the dictionary of average splines

In [ ]:
async def generateAverageBumpTest(manyBumps, averageSplineDict, id):
    # Get representative splines to be used to weed out bad runs
    [averagePrimarySpline, averageSecondarySpline] = averageSplineDict[999]
    theseBumps = manyBumps[manyBumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    # Now find the separate tests
    times = theseBumps['timestamp'].values
    startTimes = []
    endTimes = []
    for i, time in enumerate(times):
        if i == 0:
            startTimes.append(time)
            continue
        if (time - times[i-1]) > 60.0:
            startTimes.append(time)
            endTimes.append(times[i-1])
    endTimes.append(times[-1])
    numPlots = 0
    primarySplines = []
    secondarySplines = []
    for i in range(len(startTimes)):
        startTime = startTimes[i]
        endTime = endTimes[i]
        thisBump = theseBumps[(theseBumps['timestamp'] >= startTime) & (theseBumps['timestamp'] <= endTime)]
        # The pass/fail results are actually in the next test.
        lastThisBumpIndex = bumps[bumps['actuatorId']==id].last_valid_index()
        passFail = bumps.iloc[bumps.index.get_loc(lastThisBumpIndex)+1]
        if passFail[primaryBump] == 7:
            # Don't include fails
            continue
        try:
            plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
            plotEnd = plotStart + 14.0
            start = Time(plotStart, format='unix_tai', scale='tai')
            end = Time(plotEnd, format='unix_tai', scale='tai')
            forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
            times = forces['timestamp'].values
            t0 = times[0]
            times -= t0
            primaryForces = forces[primaryForce].values
            if averagePrimarySpline is not None:
                rmsError = np.sqrt(np.mean((primaryForces-averagePrimarySpline(times))**2))
            else:
                rmsError = 0.0
            if rmsError < 50.0:
                # Only include good fits in the average
                primarySpline = UnivariateSpline(times, primaryForces, s=0.0)
                primarySplines.append(primarySpline)
            if secondaryName is not None:
                if passFail[secondaryBump] == 7:
                    # Don't include fails
                    continue
                plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
                plotEnd = plotStart + 14.0
                start = Time(plotStart, format='unix_tai', scale='tai')
                end = Time(plotEnd, format='unix_tai', scale='tai')
                forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
                times = forces['timestamp'].values
                t0 = times[0]
                times -= t0
                secondaryForces = forces[secondaryForce].values
                if averageSecondarySpline is not None:
                    rmsError = np.sqrt(np.mean((secondaryForces-averageSecondarySpline(times))**2))
                else:
                    rmsError = 0.0
                if rmsError < 50.0:
                    # Only include good fits in the average
                    secondarySpline = UnivariateSpline(times, secondaryForces, s=0.0)
                    secondarySplines.append(secondarySpline)
        except:
            continue
    # Now calculate the average spline
    ts = np.linspace(0,14,5000)
    fs = np.zeros_like(ts)
    numSplines = 0
    for spline in primarySplines:
        numSplines += 1
        fs += spline(ts)
    fs /= numSplines
    averagePrimarySpline = UnivariateSpline(ts, fs)
    fs = np.zeros_like(ts)
    if secondaryName is not None:
        numSplines = 0
        for spline in secondarySplines:
            numSplines += 1
            fs += spline(ts)
        fs /= numSplines
        averageSecondarySpline = UnivariateSpline(ts, fs)
    else:
        averageSecondarySpline = None
    return [averagePrimarySpline, averageSecondarySpline]

## This cell generates the dictionary with a set of plots of the average spline

In [ ]:
# First seed the averageSplineDict with a typical bump test
# This is used to weed out bad tests
averageSplineDict = {}
averageSplineDict[999] = [None, None]
[averagePrimarySpline, averageSecondarySpline] = await generateAverageBumpTest(manyBumps, averageSplineDict, 227)
averageSplineDict[999] = [averagePrimarySpline, averageSecondarySpline]
# Now run all of the actuators
pdf = PdfPages("/home/c/cslage/u/MTM1M3/data/Average_Spline_Dict_28Apr23.pdf")
for index in range(len(FATABLE)):
    id = FATABLE[index][M1M3FATable.FATABLE_ID]
    primaryName = 'Z'
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    [averagePrimarySpline, averageSecondarySpline] = await generateAverageBumpTest(manyBumps, averageSplineDict, id)
    averageSplineDict[id] = [averagePrimarySpline, averageSecondarySpline]
    fig = plt.figure(figsize=(10,5))
    ts = np.linspace(0,14,5000)
    plt.suptitle(f"Average Spline bumps for ID = {id}")
    plt.subplot(1,2,1)
    plt.title(f"Primary  {primaryName}")
    plt.plot(ts, averagePrimarySpline(ts))
    plt.subplot(1,2,2)
    plt.title(f"Secondary   {secondaryName}")
    if averageSecondarySpline is not None:
        plt.plot(ts, averageSecondarySpline(ts))
    else:
        plt.xticks([])
        plt.yticks([])
                  
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.clf()
pdf.close()


## Pickle the dictionary for future use

In [ ]:
filename = '/home/c/cslage/u/MTM1M3/data/average_spline_dict_28apr23.pkl'
file = open(filename, 'wb')
pkl.dump(averageSplineDict, file)
file.close()

In [ ]:
filename = '/home/c/cslage/u/MTM1M3/data/average_spline_dict_28apr23.pkl'
file = open(filename, 'rb')
averageSplineDict = pkl.load(file)
file.close()

## Now plot the residuals against the average

In [ ]:
async def plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id):
    [averagePrimarySpline, averageSecondarySpline] = averageSplineDict[id]
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    # The pass/fail results are actually in the next test.
    lastThisBumpIndex = bumps[bumps['actuatorId']==id].last_valid_index()
    passFail = bumps.iloc[bumps.index.get_loc(lastThisBumpIndex)+1]
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Bump Test with Residuals. Actuator ID {id}", fontsize=18)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    primaryForces = forces[primaryForce].values
    residuals = primaryForces-averagePrimarySpline(times)
    rmsError = np.sqrt(np.mean(residuals**2))
    plotStart -= t0
    plotEnd -= t0
    plt.subplot(2,2,1)
    plt.title("Primary - Z")
    plt.plot(times, averagePrimarySpline(times), label='Average')
    plt.plot(times, primaryForces, label='Data')
    if passFail[primaryBump] == 6:
        plt.text(2.0, 350.0, "PASSED", color='g')
    elif passFail[primaryBump] == 7:
        plt.text(2.0, 350.0, "FAILED", color='r')
    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.legend()
    plt.subplot(2,2,3)
    plt.plot(times, residuals)
    if passFail[primaryBump] == 6:
        plt.text(2.0, 75.0, f"RMS = {rmsError:.2f}", color='g')
    elif passFail[primaryBump] == 7:
        plt.text(2.0, 75.0, f"RMS = {rmsError:.2f}", color='r')
    plt.xlim(plotStart, plotEnd)
    plt.ylim(-100,100)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Residuals (nt)")
    
    if secondaryName is not None:
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 14.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        secondaryForces = forces[secondaryForce].values
        residuals = secondaryForces-averageSecondarySpline(times)
        rmsError = np.sqrt(np.mean(residuals**2))
        plotStart -= t0
        plotEnd -= t0
        plt.subplot(2,2,2)
        plt.title(f"Secondary - {secondaryName}")
        plt.plot(times, averageSecondarySpline(times), label='Average')
        plt.plot(times, secondaryForces, label='Data')
        if passFail[primaryBump] == 6:
            plt.text(2.0, 350.0, "PASSED", color='g')
        elif passFail[primaryBump] == 7:
            plt.text(2.0, 350.0, "FAILED", color='r')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
        plt.legend()
        plt.subplot(2,2,4)
        plt.plot(times, residuals)
        if passFail[primaryBump] == 6:
            plt.text(2.0, 75.0, f"RMS = {rmsError:.2f}", color='g')
        elif passFail[primaryBump] == 7:
            plt.text(2.0, 75.0, f"RMS = {rmsError:.2f}", color='r')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-100,100)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Residuals (nt)")
    else:
        plt.subplot(2,2,2)
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
        plt.subplot(2,2,4)
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
id = 324
fig = plt.figure(figsize=(10,10))
await plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id)
#plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Bump_Test_Residuals_{id}_28Apr23.png")

# Now run the whole bump test

In [ ]:
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(f"/home/c/cslage/u/MTM1M3/data/Bump_Test{timestamp}.pdf")

for index in range(len(FATABLE)):
    id = FATABLE[index][M1M3FATable.FATABLE_ID]
    fig = plt.figure(figsize=(10,10))
    await plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id)
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
pdf.close()


# Now run all the actuators with the first test on the TMA

In [ ]:
# Times of bump test
start = Time("2023-04-28T18:10:00", scale='utc')
end = Time("2023-04-28T19:18:00", scale='utc')

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(f"/home/c/cslage/u/MTM1M3/data/Bump_Test{timestamp}.pdf")

for index in range(len(FATABLE)):
    id = FATABLE[index][M1M3FATable.FATABLE_ID]
    fig = plt.figure(figsize=(10,10))
    await plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id)
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
pdf.close()
